# Integrate Agenta with Agno

This notebook demonstrates how to connect **Agenta** with **Agno** for comprehensive observability and debugging of your AI agent applications.

> **What is Agenta?** [Agenta](https://agenta.ai) is an open-source LLMOps platform designed to streamline the deployment, management, and scaling of large language models. It offers comprehensive observability, testing, and deployment capabilities for AI applications.

> **What is Agno?** [Agno](https://github.com/agno-agi/agno) is a Python framework for building AI agents with tools, structured outputs, and efficient workflows. It provides a simple yet powerful interface for creating intelligent agents that can interact with external systems and perform complex tasks.

## Implementation Guide

Follow this tutorial to set up Agno with Agenta's observability platform for real-time application insights.

### Step 1: Install Required Dependencies

Install the necessary Python packages for this integration:

In [ ]:
!pip install agenta openinference-instrumentation-agno agno

**Package Descriptions:**
- `agenta`: Core SDK for Agenta's prompt engineering and observability platform
- `agno`: Framework for building AI agents with tools and structured workflows
- `openinference-instrumentation-agno`: Automatic instrumentation library for Agno operations

### Step 2: Setup and Configuration

Configure your environment and initialize the Agenta SDK:

In [ ]:
import os
import re
from itertools import permutations
import agenta as ag
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from openinference.instrumentation.agno import AgnoInstrumentor


# Load configuration from environment
os.environ["AGENTA_API_KEY"] = "your_agenta_api_key"
os.environ["AGENTA_HOST"] = (
    "https://cloud.agenta.ai"  # Optional, defaults to the Agenta cloud API
)

# Start Agenta SDK
ag.init()

**What does `ag.init()` do?**
This function initializes the Agenta SDK and sets up the necessary configuration for observability. It establishes connection to the Agenta platform, configures tracing and logging settings, and prepares the instrumentation context for your application.

### Step 3: Enable Agno Monitoring

Initialize the OpenInference Agno instrumentation to automatically capture agent operations:

In [ ]:
# Enable Agno instrumentation
AgnoInstrumentor().instrument()

### Step 4: Build Your Instrumented Agno Application

Here's a complete example showcasing a logistics dispatch agent with Agenta instrumentation:

#### Setup Logistics Data

In [ ]:
# Simulated logistics data
tracking_data = {
    "TRK10001": "In transit at Berlin Friedrichshain Distribution Center",
    "TRK10002": "Delivered on 2025-06-14 at 18:32 in Charlottenburg",
    "TRK10003": "Out for delivery — last scanned near Tempelhofer Feld",
    "TRK10004": "Held at customs near Berlin Brandenburg Airport (BER)",
    "TRK10005": "Awaiting pickup at Berlin Hauptbahnhof Parcel Station",
}

distance_matrix = {
    "Warehouse": {"A": 10, "B": 15, "C": 20},
    "A": {"Warehouse": 10, "B": 12, "C": 5},
    "B": {"Warehouse": 15, "A": 12, "C": 8},
    "C": {"Warehouse": 20, "A": 5, "B": 8},
}

driver_load = {"Alice": 2, "Bob": 3, "Charlie": 1}

#### Create Tools for the Agent

In [ ]:
# Tool: TrackingTool
class TrackingTool:
    def __init__(self):
        self.name = "TrackingTool"
        self.description = "Provides shipment status updates given a tracking ID."

    def run(self, query: str) -> str:
        match = re.search(r"\bTRK\d+\b", query.upper())
        if not match:
            return "Please provide a valid tracking ID."
        tid = match.group(0)
        status = tracking_data.get(tid)
        return f"Status for {tid}: {status}" if status else f"No information for {tid}."

In [ ]:
# Tool: RouteTool
class RouteTool:
    def __init__(self):
        self.name = "RouteTool"
        self.description = (
            "Computes the best delivery route given a start and destinations."
        )

    def run(self, query: str) -> str:
        m = re.search(r"from\s+([\w\s]+)\s+to\s+(.+)", query, re.IGNORECASE)
        if not m:
            return "Specify route as 'from <Origin> to <Dest1>, <Dest2>, ...'."
        origin = m.group(1).strip()
        dests = [d.strip() for d in re.split(r",| and ", m.group(2)) if d.strip()]

        if origin not in distance_matrix:
            return f"Unknown origin: {origin}."

        for loc in dests:
            if loc not in distance_matrix:
                return f"Unknown destination: {loc}."

        best_distance = float("inf")
        best_order = None
        for perm in permutations(dests):
            total = 0
            cur = origin

            for nxt in perm:
                total += distance_matrix[cur][nxt]
                cur = nxt

            if total < best_distance:
                best_distance = total
                best_order = perm

        route_plan = " → ".join([origin] + list(best_order)) if best_order else origin
        return f"Optimal route: {route_plan} (Total distance: {best_distance} km)"

In [ ]:
# Tool: WorkloadBalancerTool
class WorkloadBalancerTool:
    def __init__(self):
        self.name = "WorkloadBalancerTool"
        self.description = "Assigns delivery locations to the least busy driver."
        self.drivers = driver_load.copy()

    def run(self, query: str) -> str:
        m = re.search(r"deliver(?:y|ies)? to (.+)", query, re.IGNORECASE)
        if not m:
            return "Please specify delivery locations like 'deliver to A, B, C'."

        locations = [
            loc.strip() for loc in re.split(r",| and ", m.group(1)) if loc.strip()
        ]
        assignments = []
        for loc in locations:
            least_loaded = min(self.drivers, key=lambda d: self.drivers[d])
            assignments.append(f"{loc} → {least_loaded}")
            self.drivers[least_loaded] += 1

        return "Delivery assignments:\n" + "\n".join(assignments)

#### Create and Configure the Agent

In [ ]:
# Create the dispatch agent
agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are a smart dispatch assistant for a logistics team.",
    instructions=[
        "Use TrackingTool for shipment queries.",
        "Use RouteTool for route optimization.",
        "Use WorkloadBalancerTool to assign deliveries to drivers.",
        "Always return concise, formatted answers with relevant detail.",
    ],
    tools=[TrackingTool(), RouteTool(), WorkloadBalancerTool()],
    show_tool_calls=False,
)

#### Create Instrumented Handler Function

In [ ]:
@ag.instrument()
def handle_dispatch_request(query: str):
    result = agent.run(query)
    return result.content

#### Test the Logistics Agent

In [ ]:
# Test individual functionalities
print("Testing Shipment Tracking:")
tracking_response = handle_dispatch_request("Where is shipment TRK10001?")
print(tracking_response)

In [ ]:
print("\nTesting Route Optimization:")
route_response = handle_dispatch_request(
    "Find the best route from Warehouse to A, B and C"
)
print(route_response)

In [ ]:
print("\nTesting Workload Assignment:")
workload_response = handle_dispatch_request("Assign deliveries to A, B, and C")
print(workload_response)

In [ ]:
print("\nTesting Combined Multi-Tool Request:")
combined_response = handle_dispatch_request(
    "Where is shipment TRK10001? Also, find the best route from Warehouse to A, B and C, "
    "and assign deliveries to the least busy drivers."
)
print(combined_response)

### Step 5: Understanding the @ag.instrument() Decorator

The `@ag.instrument()` decorator automatically captures all input and output data from your function, enabling comprehensive observability without manual instrumentation.

**Span Type Configuration:**
Use the `spankind` parameter to categorize operations in Agenta WebUI. Available options:

- `agent` - Autonomous agent behaviors
- `chain` - Sequential processing workflows
- `workflow` - Complete application processes (default)
- `tool` - Utility and helper functions
- `embedding` - Vector embedding operations
- `query` - Search and retrieval tasks
- `completion` - Text generation operations
- `chat` - Conversational interfaces
- `rerank` - Result ordering operations

**Standard Behavior:**
By default, when `spankind` is not specified, the operation becomes a root-level span, categorized as a `workflow` in Agenta.

In [ ]:
# Example with custom span classification:
@ag.instrument(spankind="agent")
def logistics_agent_handler(query: str):
    # Agent-specific logic implementation
    pass

### Step 6: View Traces in Agenta

After running your application, access detailed execution traces through Agenta's dashboard. The observability data includes:

- Complete agent workflow execution timeline
- Agno agent initialization and tool configuration
- Tool execution sequences and decision-making processes
- LLM interactions and response generation
- Tool call chains and data flow between tools
- Performance metrics and timing analysis

<img 
    style="display: block; margin: 20px; text-align: center"
    src="./images/agenta-openinference-agno-trace.png"
    width="90%"
    alt="Agenta dashboard showing Agno application trace with detailed execution steps">

The observability interface provides insights for:
- Debug complex agent interactions and tool usage patterns
- Monitor tool execution performance and decision logic
- Analyze LLM reasoning and response quality
- Track multi-tool workflows and coordination

## Advanced Usage

### Custom Span Configuration

Customize instrumentation for different application components:

In [ ]:
@ag.instrument(spankind="workflow")
def logistics_pipeline(request: str):
    return handle_dispatch_request(request)


@ag.instrument(spankind="tool")
def external_system_integration(data: dict):
    # External system call logic
    pass


@ag.instrument(spankind="agent")
def specialized_logistics_agent(context: dict):
    # Specialized agent implementation
    return context

### Real-world Examples

#### Customer Service Agent System

In [ ]:
# Customer service data
customer_db = {
    "CUST001": {"name": "John Doe", "tier": "Premium", "issues": []},
    "CUST002": {"name": "Jane Smith", "tier": "Standard", "issues": ["billing"]},
}

ticket_system = {
    "TKT001": {"customer": "CUST001", "status": "open", "priority": "high"},
    "TKT002": {"customer": "CUST002", "status": "resolved", "priority": "medium"},
}


class CustomerLookupTool:
    def __init__(self):
        self.name = "CustomerLookupTool"
        self.description = "Retrieves customer information by ID."

    def run(self, query: str) -> str:
        match = re.search(r"\bCUST\d+\b", query.upper())
        if not match:
            return "Please provide a valid customer ID."

        cust_id = match.group(0)
        customer = customer_db.get(cust_id)
        if customer:
            return f"Customer {cust_id}: {customer['name']}, Tier: {customer['tier']}"
        return f"No customer found for {cust_id}."


class TicketManagementTool:
    def __init__(self):
        self.name = "TicketManagementTool"
        self.description = "Manages support tickets and escalations."

    def run(self, query: str) -> str:
        if "create ticket" in query.lower():
            return "New ticket TKT003 created successfully."

        elif "escalate" in query.lower():
            return "Ticket escalated to senior support team."

        match = re.search(r"\bTKT\d+\b", query.upper())
        if match:
            ticket_id = match.group(0)
            ticket = ticket_system.get(ticket_id)
            if ticket:
                return f"Ticket {ticket_id}: Status: {ticket['status']}, Priority: {ticket['priority']}"

        return "Please specify a valid ticket action or ID."


customer_service_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are a customer service agent helping with inquiries and ticket management.",
    instructions=[
        "Use CustomerLookupTool to find customer information.",
        "Use TicketManagementTool to manage support tickets.",
        "Always be helpful and professional in your responses.",
    ],
    tools=[CustomerLookupTool(), TicketManagementTool()],
    show_tool_calls=False,
)


@ag.instrument(spankind="workflow")
def handle_customer_request(query: str):
    result = customer_service_agent.run(query)
    return result.content

#### Test Customer Service Agent

In [ ]:
# Test customer service functionality
customer_response = handle_customer_request(
    "Look up customer CUST001 and check ticket TKT001"
)
print("Customer Service Response:", customer_response)

#### E-commerce Order Management System

In [ ]:
# E-commerce data
inventory = {
    "PROD001": {"name": "Laptop", "stock": 5, "price": 999.99},
    "PROD002": {"name": "Mouse", "stock": 25, "price": 29.99},
    "PROD003": {"name": "Keyboard", "stock": 0, "price": 79.99},
}

orders = {
    "ORD001": {"customer": "CUST001", "items": ["PROD001"], "status": "shipped"},
    "ORD002": {"customer": "CUST002", "items": ["PROD002"], "status": "processing"},
}


class InventoryTool:
    def __init__(self):
        self.name = "InventoryTool"
        self.description = "Checks product availability and stock levels."

    def run(self, query: str) -> str:
        match = re.search(r"\bPROD\d+\b", query.upper())
        if not match:
            return "Please provide a valid product ID."

        prod_id = match.group(0)
        product = inventory.get(prod_id)
        if product:
            status = "In Stock" if product["stock"] > 0 else "Out of Stock"
            return f"Product {prod_id}: {product['name']}, Stock: {product['stock']}, Price: ${product['price']}, Status: {status}"

        return f"Product {prod_id} not found."


class OrderManagementTool:
    def __init__(self):
        self.name = "OrderManagementTool"
        self.description = "Manages order status and updates."

    def run(self, query: str) -> str:
        match = re.search(r"\bORD\d+\b", query.upper())
        if not match:
            return "Please provide a valid order ID."

        order_id = match.group(0)
        order = orders.get(order_id)
        if order:
            return f"Order {order_id}: Customer: {order['customer']}, Items: {order['items']}, Status: {order['status']}"
        return f"Order {order_id} not found."


ecommerce_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are an e-commerce assistant helping with inventory and order management.",
    instructions=[
        "Use InventoryTool to check product availability.",
        "Use OrderManagementTool to track orders.",
        "Provide clear and helpful information about products and orders.",
    ],
    tools=[InventoryTool(), OrderManagementTool()],
    show_tool_calls=False,
)


@ag.instrument(spankind="agent")
def handle_ecommerce_request(query: str):
    result = ecommerce_agent.run(query)
    return result.content

#### Test E-commerce Agent

In [ ]:
# Test e-commerce functionality
ecommerce_response = handle_ecommerce_request(
    "Check availability of PROD001 and status of order ORD001"
)
print("E-commerce Response:", ecommerce_response)

#### Financial Trading Assistant

In [ ]:
# Financial data
portfolio = {
    "AAPL": {"shares": 100, "avg_price": 150.00, "current_price": 175.50},
    "GOOGL": {"shares": 50, "avg_price": 2800.00, "current_price": 2950.00},
    "TSLA": {"shares": 25, "avg_price": 800.00, "current_price": 725.00},
}

market_data = {
    "AAPL": {"price": 175.50, "change": "+2.5%", "volume": "45M"},
    "GOOGL": {"price": 2950.00, "change": "+1.8%", "volume": "12M"},
    "TSLA": {"price": 725.00, "change": "-3.2%", "volume": "28M"},
}


class PortfolioTool:
    def __init__(self):
        self.name = "PortfolioTool"
        self.description = "Provides portfolio information and performance analysis."

    def run(self, query: str) -> str:
        if "portfolio" in query.lower():
            total_value = sum(
                stock["shares"] * stock["current_price"] for stock in portfolio.values()
            )
            return f"Portfolio Total Value: ${total_value:,.2f}"

        match = re.search(r"\b[A-Z]{2,5}\b", query.upper())
        if match:
            symbol = match.group(0)
            if symbol in portfolio:
                stock = portfolio[symbol]
                current_value = stock["shares"] * stock["current_price"]
                return f"{symbol}: {stock['shares']} shares @ ${stock['current_price']}, Value: ${current_value:,.2f}"

        return "Please specify a valid stock symbol or ask about your portfolio."


class MarketDataTool:
    def __init__(self):
        self.name = "MarketDataTool"
        self.description = "Provides real-time market data and stock information."

    def run(self, query: str) -> str:
        match = re.search(r"\b[A-Z]{2,5}\b", query.upper())
        if not match:
            return "Please provide a valid stock symbol."

        symbol = match.group(0)
        if symbol in market_data:
            data = market_data[symbol]
            return f"{symbol}: ${data['price']}, Change: {data['change']}, Volume: {data['volume']}"
        return f"No market data available for {symbol}."


trading_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are a financial trading assistant providing portfolio and market analysis.",
    instructions=[
        "Use PortfolioTool to analyze portfolio holdings.",
        "Use MarketDataTool to get current market information.",
        "Provide clear financial insights and recommendations.",
    ],
    tools=[PortfolioTool(), MarketDataTool()],
    show_tool_calls=False,
)


@ag.instrument(spankind="workflow")
def handle_trading_request(query: str):
    result = trading_agent.run(query)
    return result.content

#### Test Trading Agent

In [ ]:
# Test trading functionality
trading_response = handle_trading_request(
    "Show me my AAPL holdings and current market data for GOOGL"
)
print("Trading Response:", trading_response)

### Additional Testing Examples

In [ ]:
# Test portfolio overview
portfolio_overview = handle_trading_request("What is my total portfolio value?")
print("Portfolio Overview:", portfolio_overview)

In [ ]:
# Test customer service ticket creation
ticket_creation = handle_customer_request("Create a new ticket for billing issues")
print("Ticket Creation:", ticket_creation)

In [ ]:
# Test inventory check for out of stock item
inventory_check = handle_ecommerce_request("Check if PROD003 is available for purchase")
print("Inventory Check:", inventory_check)

## Next Steps

For more detailed information about Agenta's observability features and advanced configuration options, visit the [Agenta Observability SDK Documentation](/observability/observability-sdk).